In [ ]:
# Note: It is important to not call any other function such as store.list_dir
# after creating the store and before creating the group, as this will lead to an error
# "got Future <Future pending> attached to a different loop"

# async for item in store.list_dir(""):
#     print(item)

In [ ]:
# mamba install h5py zarr --channel conda-forge

# There is a bug with the newest versions of botocore (1.36 and 1.37)
# mamba install botocore=1.35

In [ ]:
import os

import h5py
import zarr
from s3fs import S3FileSystem

In [ ]:
path = "qrage/sessions"

In [ ]:
fs_sync = S3FileSystem(
    anon=False,
    endpoint_url="https://s3.fz-juelich.de",
    key=os.getenv("AWS_ACCESS_KEY_ID"),
    secret=os.getenv("AWS_SECRET_ACCESS_KEY"),
)

In [ ]:
fs_async = S3FileSystem(
    anon=False,
    endpoint_url="https://s3.fz-juelich.de",
    key=os.getenv("AWS_ACCESS_KEY_ID"),
    secret=os.getenv("AWS_SECRET_ACCESS_KEY"),
    asynchronous=True,
)

In [ ]:
sessions = fs_sync.ls(path, refresh=True)
sessions.sort()

In [ ]:
for session in sessions:
    print(session)

    """
    # Delete all zarr files
    if fs_sync.isdir(f'{session}/preproc.zarr'):
        print(f'Deleting {session}/preproc.zarr ...')
        fs_sync.rm(f'{session}/preproc.zarr', recursive=True)
    """

    """
    # Delete all h5 files
    if fs_sync.isdir(f'{session}/preproc'):
        print(f'Deleting {session}/preproc ...')
        fs_sync.rm(f'{session}/preproc', recursive=True)
    """

    if not fs_sync.isdir(f"{session}/preproc.zarr"):
        store = zarr.storage.FsspecStore(
            fs_async,
            path=f"{session}/preproc.zarr/preproc.zarr",
        )

        root = zarr.create_group(
            store=store,
            overwrite=True,
        )

        try:
            # with fs_sync.open(f'{folder}/mz_me_mpnrage3d_grappa.h5', mode='rb') as f:
            # with h5py.File(f, mode='r', libver='latest') as hf:
            with h5py.File(
                f"/home/projects/{session}/preproc/mz_me_mpnrage3d_grappa.h5",
                mode="r",
                libver="latest",
            ) as hf:
                for key in hf.keys():
                    print(key, hf[key].shape, hf[key].chunks, hf[key].dtype)
                    array = root.create_array(
                        name=key,
                        shape=hf[key].shape,
                        chunks=hf[key].chunks,
                        dtype=hf[key].dtype,
                        overwrite=True,
                    )
                    array[:] = hf[key][:]

        except:
            print(f"{session} failed.")